In [1]:
# imports
import os
import pandas as pd
import numpy as np
import statsmodels.api as sm
import statsmodels.formula.api as smf

from econml.dml import LinearDML
# from econml.dr import LinearDRLearner
from econml.drlearner import LinearDRLearner
from econml.cate_interpreter import SingleTreeCateInterpreter

from sklearn.linear_model import LogisticRegressionCV, LinearRegression
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.preprocessing import StandardScaler

# some features will be standardized
scaler = StandardScaler()

import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline


In [5]:
[f for f in os.listdir('/Users/boyuliu/pyprojects/Joann/Joann-Thailand-Project/notebooks/datasets/new_dataset/') if 
f.endswith('20210301.csv')]

['regression_data_wv_cases1_20210301.csv',
 'regression_data_wv_cases1_causal_ma_detrend_20210301.csv',
 'regression_data_wv_cases1_causal_ma_detrend_12w_20210301.csv',
 'regression_data_large_p_wv_cases1_20210301.csv',
 'regression_data_wv_cases2_20210301.csv',
 'regression_data_wv_cases1_causal_ma_detrend_8w_20210301.csv',
 'regression_data_wv_cases4_20210301.csv',
 'regression_data_wv_cases3_20210301.csv']

In [6]:
data_dir = '/Users/boyuliu/pyprojects/Joann/Joann-Thailand-Project/notebooks/datasets/new_dataset/'
wv_data_file = 'wv_cases1'

reg_data = pd.read_csv(data_dir + 'regression_data_%s_causal_ma_detrend_20210301.csv' % wv_data_file)
reg_data_notnull = reg_data.loc[pd.notnull(reg_data['demand_shock'])].reset_index()
reg_data.head()

,province,year_week,total_demand,perc_abuse,wv_count,ex_rate,fake_date,month,quarter,ex_rate_diff,...,demand_shock_plus_8,demand_shock_minus_1,demand_shock_minus_2,demand_shock_minus_3,demand_shock_minus_4,demand_shock_minus_5,demand_shock_minus_6,demand_shock_minus_7,demand_shock_minus_8,yr_wk_float
0,Ang Thong,2018-02,0.0,0.0,0.0,5.00602,2018-01-08,2018-01,2018-1,-0.032813,...,NaN,NaN,0.000000,0.983368,-0.245842,-0.245842,-0.245842,1.229210,-0.368763,2018.02
1,Ang Thong,2018-03,0.0,0.0,0.0,5.02540,2018-01-15,2018-01,2018-1,0.019380,...,NaN,0.000000,0.983368,-0.245842,-0.245842,-0.245842,1.229210,-0.368763,-0.368763,2018.03
2,Ang Thong,2018-04,0.0,0.0,0.0,5.03024,2018-01-22,2018-01,2018-1,0.004840,...,NaN,0.983368,-0.245842,-0.245842,-0.245842,1.229210,-0.368763,-0.368763,-0.368763,2018.04
3,Ang Thong,2018-05,10.0,0.0,0.0,5.03048,2018-01-29,2018-01,2018-1,0.000240,...,NaN,-0.245842,-0.245842,-0.245842,1.229210,-0.368763,-0.368763,-0.368763,-0.368763,2018.05
4,Ang Thong,2018-06,0.0,0.0,0.0,5.08050,2018-02-05,2018-02,2018-1,0.050020,...,NaN,-0.245842,-0.245842,1.229210,-0.368763,-0.368763,-0.368763,-0.368763,0.000000,2018.06


<h1>demand shock</h1>

In [7]:
md = smf.mixedlm("perc_abuse ~ demand_shock", reg_data_notnull, groups=reg_data_notnull["province"])
mdf = md.fit()
print(mdf.summary())

         Mixed Linear Model Regression Results
Model:            MixedLM Dependent Variable: perc_abuse
No. Observations: 6438    Method:             REML      
No. Groups:       58      Scale:              0.0327    
Min. group size:  111     Likelihood:         1791.3384 
Max. group size:  111     Converged:          Yes       
Mean group size:  111.0                                 
--------------------------------------------------------
                Coef. Std.Err.   z   P>|z| [0.025 0.975]
--------------------------------------------------------
Intercept       0.056    0.009 6.404 0.000  0.039  0.073
demand_shock    0.000    0.002 0.121 0.904 -0.004  0.004
Group Var       0.004    0.005                          



/usr/local/lib/python3.7/site-packages/statsmodels/regression/mixed_linear_model.py:2094: ConvergenceWarning: The MLE may be on the boundary of the parameter space.
  warnings.warn(msg, ConvergenceWarning)


In [8]:
mdf.params

Intercept       0.055914
demand_shock    0.000246
Group Var       0.126324
dtype: float64

In [9]:
reg_data_20 = pd.read_csv(data_dir + 'regression_data_wv_cases1_causal_detrend_province_20_20210209.csv')
reg_data_20 = reg_data_20.loc[pd.notnull(reg_data_20['demand_shock'])].reset_index()

md = smf.mixedlm("perc_abuse ~ demand_shock", reg_data_20, groups=reg_data_20["province"])
mdf = md.fit()
print(mdf.summary())

         Mixed Linear Model Regression Results
Model:            MixedLM Dependent Variable: perc_abuse
No. Observations: 3552    Method:             REML      
No. Groups:       32      Scale:              0.0537    
Min. group size:  111     Likelihood:         110.9019  
Max. group size:  111     Converged:          Yes       
Mean group size:  111.0                                 
--------------------------------------------------------
                Coef. Std.Err.   z   P>|z| [0.025 0.975]
--------------------------------------------------------
Intercept       0.095    0.012 8.110 0.000  0.072  0.118
demand_shock    0.009    0.004 2.445 0.015  0.002  0.016
Group Var       0.004    0.005                          



/usr/local/lib/python3.7/site-packages/statsmodels/regression/mixed_linear_model.py:2094: ConvergenceWarning: The MLE may be on the boundary of the parameter space.
  warnings.warn(msg, ConvergenceWarning)


In [10]:
reg_data_40 = pd.read_csv(data_dir + 'regression_data_wv_cases1_causal_detrend_province_40_20210209.csv')
reg_data_40 = reg_data_40.loc[pd.notnull(reg_data_40['demand_shock'])].reset_index()


md = smf.mixedlm("perc_abuse ~ demand_shock", reg_data_40, groups=reg_data_40["province"])
mdf = md.fit()
print(mdf.summary())

         Mixed Linear Model Regression Results
Model:            MixedLM Dependent Variable: perc_abuse
No. Observations: 2664    Method:             REML      
No. Groups:       24      Scale:              0.0651    
Min. group size:  111     Likelihood:         -168.3707 
Max. group size:  111     Converged:          Yes       
Mean group size:  111.0                                 
--------------------------------------------------------
               Coef. Std.Err.   z    P>|z| [0.025 0.975]
--------------------------------------------------------
Intercept      0.119    0.012 10.083 0.000  0.096  0.142
demand_shock   0.012    0.005  2.619 0.009  0.003  0.021
Group Var      0.003    0.004                           



/usr/local/lib/python3.7/site-packages/statsmodels/regression/mixed_linear_model.py:2094: ConvergenceWarning: The MLE may be on the boundary of the parameter space.
  warnings.warn(msg, ConvergenceWarning)


# only province fixed effects

In [11]:
x_columns = ['demand_shock']
y_column = 'perc_abuse'

# reg_data_notnull = reg_data.loc[pd.notnull(reg_data['demand_shock'])].reset_index()
province_dummies = pd.get_dummies(reg_data_notnull['province'])
X = reg_data_notnull[x_columns]
X = pd.concat([X, province_dummies], axis=1) 
print(X.shape)
# X = sm.add_constant(X)
Y = reg_data_notnull[y_column]

model = sm.OLS(Y, X, hasconst=False)
results = model.fit()
# results = model.fit(cov_type='cluster', cov_kwds=province_dummies.columns)
# print(results.summary())
print(results.get_robustcov_results().summary())
# # save results for summary table
# effect = results.params['treated']
# se = results.bse['treated']
# if print_summary:
#     print(results.summary())
# else:
#     print('treatment effect is %s, standard error is %s' % (effect, se))

(6438, 59)
                                 OLS Regression Results                                
Dep. Variable:             perc_abuse   R-squared (uncentered):                   0.188
Model:                            OLS   Adj. R-squared (uncentered):              0.180
Method:                 Least Squares   F-statistic:                              13.06
Date:                Fri, 12 Mar 2021   Prob (F-statistic):                   1.73e-116
Time:                        01:47:39   Log-Likelihood:                          1907.7
No. Observations:                6438   AIC:                                     -3697.
Df Residuals:                    6379   BIC:                                     -3298.
Df Model:                          59                                                  
Covariance Type:                  HC1                                                  
                               coef    std err          t      P>|t|      [0.025      0.975]
----------------

In [23]:
x_columns = ['demand_shock']
y_column = 'perc_abuse'

# reg_data_notnull = reg_data.loc[pd.notnull(reg_data['demand_shock'])].reset_index()
province_dummies = pd.get_dummies(reg_data_40['province'])
X = reg_data_40[x_columns]
X = pd.concat([X, province_dummies], axis=1) 
print(X.shape)
# X = sm.add_constant(X)
Y = reg_data_40[y_column]

model = sm.OLS(Y, X, hasconst=False)
# results = model.fit()
# results = model.fit(cov_type='cluster', cov_kwds=province_dummies.columns)
results = model.fit(cov_type='cluster', cov_kwds={"groups":reg_data_40['province']})
# print(results.summary())
print(results.get_robustcov_results().summary())
# # save results for summary table
# effect = results.params['treated']
# se = results.bse['treated']
# if print_summary:
#     print(results.summary())
# else:
#     print('treatment effect is %s, standard error is %s' % (effect, se))

(2664, 25)
                                 OLS Regression Results                                
Dep. Variable:             perc_abuse   R-squared (uncentered):                   0.217
Model:                            OLS   Adj. R-squared (uncentered):              0.210
Method:                 Least Squares   F-statistic:                              29.18
Date:                Thu, 18 Mar 2021   Prob (F-statistic):                   2.00e-120
Time:                        00:37:38   Log-Likelihood:                         -128.03
No. Observations:                2664   AIC:                                      306.1
Df Residuals:                    2639   BIC:                                      453.3
Df Model:                          25                                                  
Covariance Type:                  HC1                                                  
                               coef    std err          z      P>|z|      [0.025      0.975]
----------------

# province interaction terms as well

### since AIC/BIC and R-squared are worse here, we should not include interaction terms

In [12]:
x_columns = ['demand_shock']
y_column = 'perc_abuse'

reg_data_notnull = reg_data.loc[pd.notnull(reg_data['demand_shock'])].reset_index()
province_dummies = pd.get_dummies(reg_data_notnull['province']).iloc[:,1:]
interaction_terms = pd.DataFrame(province_dummies.values * reg_data_notnull[x_columns].values)
interaction_terms.columns = [prov.replace(' ', '_')+'_beta' for prov in province_dummies.columns]
X = reg_data_notnull[x_columns]
X = pd.concat([X, province_dummies, interaction_terms], axis=1) 
X = sm.add_constant(X)
Y = reg_data_notnull[y_column]

model = sm.OLS(Y, X, hasconst=True)
results = model.fit()
# print(results.summary())
print(results.get_robustcov_results().summary())

/usr/local/lib/python3.7/site-packages/numpy/core/fromnumeric.py:2608: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return ptp(axis=axis, out=out, **kwargs)


                            OLS Regression Results                            
Dep. Variable:             perc_abuse   R-squared:                       0.126
Model:                            OLS   Adj. R-squared:                  0.111
Method:                 Least Squares   F-statistic:                     9.092
Date:                Fri, 12 Mar 2021   Prob (F-statistic):          4.79e-105
Time:                        01:50:14   Log-Likelihood:                 1937.5
No. Observations:                6438   AIC:                            -3643.
Df Residuals:                    6322   BIC:                            -2858.
Df Model:                         115                                         
Covariance Type:                  HC1                                         
                                    coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------------------
const         

/usr/local/lib/python3.7/site-packages/statsmodels/base/model.py:1752: ValueWarning: covariance of constraints does not have full rank. The number of constraints is 115, but rank is 86
  'rank is %d' % (J, J_), ValueWarning)


In [13]:
x_columns = ['demand_shock']
y_column = 'perc_abuse'

reg_data_notnull = reg_data.loc[pd.notnull(reg_data['demand_shock'])].reset_index()
province_dummies = pd.get_dummies(reg_data_notnull['province'])
interaction_terms = pd.DataFrame(province_dummies.iloc[:,1:].values * reg_data_notnull[x_columns].values)
interaction_terms.columns = [prov.replace(' ', '_')+'_beta' for prov in province_dummies.columns[1:]]
X = reg_data_notnull[x_columns]
X = pd.concat([X, province_dummies, interaction_terms], axis=1) 
Y = reg_data_notnull[y_column]

model = sm.OLS(Y, X, hasconst=True)
results = model.fit()
# print(results.summary())
print(results.get_robustcov_results().summary())

                            OLS Regression Results                            
Dep. Variable:             perc_abuse   R-squared:                       0.126
Model:                            OLS   Adj. R-squared:                  0.111
Method:                 Least Squares   F-statistic:                       nan
Date:                Fri, 12 Mar 2021   Prob (F-statistic):                nan
Time:                        01:53:25   Log-Likelihood:                 1937.5
No. Observations:                6438   AIC:                            -3643.
Df Residuals:                    6322   BIC:                            -2858.
Df Model:                         115                                         
Covariance Type:                  HC1                                         
                                    coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------------------
demand_shock  

In [17]:
x_columns = ['demand_shock']
y_column = 'perc_abuse'

province_dummies = pd.get_dummies(reg_data_40['province'])
interaction_terms = pd.DataFrame(province_dummies.iloc[:,1:].values * reg_data_40[x_columns].values)
interaction_terms.columns = [prov.replace(' ', '_')+'_beta' for prov in province_dummies.columns[1:]]
X = reg_data_40[x_columns]
X = pd.concat([X, province_dummies, interaction_terms], axis=1) 
Y = reg_data_40[y_column]

model = sm.OLS(Y, X, hasconst=True)
results = model.fit()
# print(results.summary())
print(results.get_robustcov_results().summary())

                            OLS Regression Results                            
Dep. Variable:             perc_abuse   R-squared:                       0.060
Model:                            OLS   Adj. R-squared:                  0.043
Method:                 Least Squares   F-statistic:                       nan
Date:                Thu, 18 Mar 2021   Prob (F-statistic):                nan
Time:                        00:31:31   Log-Likelihood:                -114.73
No. Observations:                2664   AIC:                             325.5
Df Residuals:                    2616   BIC:                             608.1
Df Model:                          47                                         
Covariance Type:                  HC1                                         
                                    coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------------------
demand_shock  

In [14]:
# print(sm.OLS.from_formula('perc_abuse ~ demand_shock + C(province)',
#                           data=reg_data_notnull).fit(cov_type="cluster",cov_kwds={"groups":'province'}).summary())
print(sm.OLS.from_formula('perc_abuse ~ demand_shock + C(province)',
                          data=reg_data_notnull).fit().summary())

                            OLS Regression Results                            
Dep. Variable:             perc_abuse   R-squared:                       0.118
Model:                            OLS   Adj. R-squared:                  0.110
Method:                 Least Squares   F-statistic:                     14.76
Date:                Fri, 12 Mar 2021   Prob (F-statistic):          1.88e-132
Time:                        01:53:43   Log-Likelihood:                 1907.7
No. Observations:                6438   AIC:                            -3697.
Df Residuals:                    6379   BIC:                            -3298.
Df Model:                          58                                         
Covariance Type:            nonrobust                                         
                                              coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------------------------

In [15]:
print(sm.OLS.from_formula('perc_abuse ~ demand_shock + C(province)',
                          data=reg_data_notnull).fit(
    cov_type="cluster",cov_kwds={"groups":reg_data_notnull['province']}).summary())

                            OLS Regression Results                            
Dep. Variable:             perc_abuse   R-squared:                       0.118
Model:                            OLS   Adj. R-squared:                  0.110
Method:                 Least Squares   F-statistic:                     496.3
Date:                Fri, 12 Mar 2021   Prob (F-statistic):           8.20e-30
Time:                        01:54:37   Log-Likelihood:                 1907.7
No. Observations:                6438   AIC:                            -3697.
Df Residuals:                    6379   BIC:                            -3298.
Df Model:                          58                                         
Covariance Type:              cluster                                         
                                              coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------------------------------

/usr/local/lib/python3.7/site-packages/statsmodels/base/model.py:1752: ValueWarning: covariance of constraints does not have full rank. The number of constraints is 58, but rank is 1
  'rank is %d' % (J, J_), ValueWarning)


<h1>Total demand</h1>

In [22]:
x_columns = ['total_demand']
y_column = 'perc_abuse'

province_dummies = pd.get_dummies(reg_data_40['province'])
interaction_terms = pd.DataFrame(province_dummies.iloc[:,1:].values * reg_data_40[x_columns].values)
interaction_terms.columns = [prov.replace(' ', '_')+'_beta' for prov in province_dummies.columns[1:]]
X = reg_data_40[x_columns]
X = pd.concat([X, province_dummies, interaction_terms], axis=1) 
Y = reg_data_40[y_column]

model = sm.OLS(Y, X, hasconst=True)
# results = model.fit()
results = model.fit(cov_type='cluster', cov_kwds={"groups":reg_data_40['province']})
# print(results.summary())
print(results.get_robustcov_results().summary())

                            OLS Regression Results                            
Dep. Variable:             perc_abuse   R-squared:                       0.057
Model:                            OLS   Adj. R-squared:                  0.040
Method:                 Least Squares   F-statistic:                       nan
Date:                Thu, 18 Mar 2021   Prob (F-statistic):                nan
Time:                        00:36:21   Log-Likelihood:                -118.34
No. Observations:                2664   AIC:                             332.7
Df Residuals:                    2616   BIC:                             615.3
Df Model:                          47                                         
Covariance Type:                  HC1                                         
                                    coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------------------------
total_demand  

In [21]:
x_columns = ['total_demand']
y_column = 'perc_abuse'

# reg_data_notnull = reg_data.loc[pd.notnull(reg_data['demand_shock'])].reset_index()
province_dummies = pd.get_dummies(reg_data_40['province'])
X = reg_data_40[x_columns]
X = pd.concat([X, province_dummies], axis=1) 
print(X.shape)
# X = sm.add_constant(X)
Y = reg_data_40[y_column]

model = sm.OLS(Y, X, hasconst=False)
# results = model.fit()
results = model.fit(cov_type='cluster', cov_kwds={"groups":reg_data_40['province']})
# print(results.summary())
print(results.get_robustcov_results().summary())

(2664, 25)
                                 OLS Regression Results                                
Dep. Variable:             perc_abuse   R-squared (uncentered):                   0.216
Model:                            OLS   Adj. R-squared (uncentered):              0.209
Method:                 Least Squares   F-statistic:                              29.20
Date:                Thu, 18 Mar 2021   Prob (F-statistic):                   1.61e-120
Time:                        00:36:02   Log-Likelihood:                         -129.30
No. Observations:                2664   AIC:                                      308.6
Df Residuals:                    2639   BIC:                                      455.8
Df Model:                          25                                                  
Covariance Type:                  HC1                                                  
                               coef    std err          z      P>|z|      [0.025      0.975]
----------------